## Vision Bootcamp - Day 2: Practical Application

### Description of the Practical App.

The aim of this practical application is to integrate an object detection method into a specific part of the code to identify objects in images captured by the Basler camera, by connecting to a live camera or by loading an image from a file. The YOLO detection model will be used to detect and highlight the INTEMAC logo in the captured image, ensuring accurate and efficient identification.

## Importing Required Libraries

In this first step, we import the necessary libraries that will be used throughout this practical app.:

- **OpenCV (`cv2`)**: This is the main library we’ll use for image processing tasks, such as reading, displaying, and manipulating images.
- **OS (`os`)**: This helps with file and directory operations, allowing us to load and save images from specific locations on your computer.
- **Ultralytics (`YOLO`)**: This library provides real-time object detection and image segmentation capabilities. It enables us to use the YOLO model for accurate and efficient object detection.
- **Utilities.Camera (`Basler_Cls`)**: The `Basler_Cls` class is designed to interface with Basler cameras using the `pypylon` library. It allows us to configure the camera, capture images, and release resources when done.

This is the foundation for a smooth YOLO-based object detection workflow, and with these tools, we can start detecting objects in images using AI in an interactive and efficient way!

In [ ]:
# Importing required libraries

# OpenCV library for computer vision tasks
import cv2

# OS module for file handling and accessing directories
import os

# Ultralytics (Real-time object detection and image segmentation 
# model) [pip install ultralytics]
from ultralytics import YOLO

# Library to work with Basler cameras
from Utilities.Camera import Basler_Cls

# Simple Object Detection Using Basler Camera and VMR Lighting

## Practical Setup

- **Camera Model**: Basler a2A2448-23gcPRO GigE Camera 
- **Lighting**: VMR-11566 Multi Angle Ring Light

This section demonstrates how to configure a **Basler camera (a2A2448-23gcPRO)** with custom settings, capture an image, and perform object detection using YOLO. The system is equipped with the **VMR-11566W lighting** for optimal illumination in **stand no. 1**. 

In [ ]:
# Custom camera configuration
custom_cfg = {
    'exposure_time': 5000,
    'gain': 15,
    'balance_ratios': {'Red': 1.3, 'Green': 1.0, 'Blue': 1.1},
    'pixel_format': 'BayerRG8'
}

# Initialize and configure the Basler camera
Basler_Cam_Id_1 = Basler_Cls(config=custom_cfg)

# Capture a single image
img_raw = Basler_Cam_Id_1.Capture()
if img_raw is not None:
    # Convert the image from BGR to RGB color format
    img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
else:
    raise ValueError('No image captured!')

# Release the camera resources
del Basler_Cam_Id_1

# Copy the original image to draw contours and bounding boxes on it
img_registred_1 = img_rgb.copy()

# Add object detection code here...

# Define output image path
output_path = os.path.join(f'../Data/Stand_1/Eval/', 'Image_X_result.png')

# Save the image with bounding boxes
cv2.imwrite(output_path, img_raw)

print(f'Result saved at: {output_path}')

This section demonstrates how to load the image captured by the camera on **stand no. 1** and perform object detection using YOLO.  

In [ ]:
# Locate the path to the project folder
project_folder = os.getcwd().split('INTEMAC_Vision_Bootcamp')[0] + 'INTEMAC_Vision_Bootcamp'

# Define file path for the image location
file_path = '../Data/Stand_1/Dataset_v1/images/test/'

# Create the directory if it doesn't already exist
os.makedirs(file_path, exist_ok=True)

# Define the path to the image file
img_path_in = os.path.join(file_path, 'Image_31.png')

# Load the image using OpenCV
img_raw = cv2.imread(img_path_in)

# Check if the image exists
if img_raw is None:
    raise FileNotFoundError(f'Image not found at {img_path_in}')

# Convert the image from BGR to RGB color format
img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)

# Get image dimensions
img_height, img_width, _ = img_rgb.shape

# Load a pre-trained custom YOLO model.
model = YOLO(f'{project_folder}/YOLO/Results/Stand_1_Dataset_v1/train_fb_True/weights/best.pt')

# Predict (test) the model on a test dataset.
results = model.predict(source=img_path_in, imgsz=640, conf=0.5, iou=0.7)

# Define a dictionary for class_id to color mapping
class_colors = {0: (0, 255, 255), 1: (255, 0, 0)}  # Yellow and Blue in BGR

# If objects are detected in the image
if results[0].boxes.shape[0] >= 1:
    class_id = results[0].boxes.cls.cpu().numpy(); b_box = results[0].boxes.xywhn.cpu().numpy()
    conf = results[0].boxes.conf.cpu().numpy()

    for _, (class_id_i, b_box_i, conf_i) in enumerate(zip(class_id, b_box, conf)):
        x_center, y_center, width, height = b_box_i

        # Convert YOLO coordinates to pixel coordinates
        x_c = int(x_center * img_width); y_c = int(y_center * img_height)
        w = int(width * img_width); h = int(height * img_height)

        # Calculate the top-left corner of the bounding box
        x_top_left = int(x_c - w / 2); y_top_left = int(y_c - h / 2)
        x_bottom_right = int(x_c + w / 2); y_bottom_right = int(y_c + h / 2)

        # Get the bounding box color
        class_id_color = class_colors.get(class_id_i, (0, 255, 0))  # Green as default

        # Draw the bounding box
        cv2.rectangle(img_raw, (x_top_left, y_top_left), (x_bottom_right, y_bottom_right), class_id_color, 2)

        # Put class ID and confidence score
        label = f'Class {int(class_id_i)}: {conf_i:.2f}'
        cv2.putText(img_raw, label, (x_top_left, y_top_left - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, class_id_color, 2)

# Define output image path
output_path = os.path.join(f'../Data/Stand_1/Eval/', 'Image_31_result.png')

# Save the image with bounding boxes
cv2.imwrite(output_path, img_raw)

print(f'Result saved at: {output_path}')

# Simple Object Detection Using Basler Camera and EFFI-FD Lighting

## Practical Setup

- **Camera Model**: Basler a2A1920-51gcPRO GigE Camera
- **Lighting**: EFFI-FD-200-200-000 High-Power Flat Light

This section demonstrates how to configure a **Basler camera (a2A1920-51gcPRO)** with custom settings, capture an image, and perform object detection method using YOLO. The system is equipped with the **EFFI-FD-200-200-000 lighting** for optimal illumination in **stand no. 2**. 

In [ ]:
# Custom camera configuration
custom_cfg = {
    'exposure_time': 1000,
    'gain': 15,
    'balance_ratios': {'Red': 1.1, 'Green': 1.0, 'Blue': 1.3},
    'pixel_format': 'BayerRG8'
}

# Initialize and configure the Basler camera
Basler_Cam_Id_2 = Basler_Cls(config=custom_cfg)

# Capture a single image
img_raw = Basler_Cam_Id_2.Capture()
if img_raw is not None:
    # Convert the image from BGR to RGB color format
    img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
else:
    raise ValueError('No image captured!')

# Release the camera resources
del Basler_Cam_Id_2

# Copy the original image to draw contours and bounding boxes on it
img_registred_2 = img_rgb.copy()

# Define output image path
output_path = os.path.join(f'../Data/Stand_2/Eval/', 'Image_X_result.png')

# Save the image with bounding boxes
cv2.imwrite(output_path, img_raw)

print(f'Result saved at: {output_path}')

This section demonstrates how to load the image captured by the camera on **stand no. 2** and perform object detection using YOLO.  

In [ ]:
# Locate the path to the project folder
project_folder = os.getcwd().split('INTEMAC_Vision_Bootcamp')[0] + 'INTEMAC_Vision_Bootcamp'

# Define file path for the image location
file_path = '../Data/Stand_2/Dataset_v1/images/test/'

# Create the directory if it doesn't already exist
os.makedirs(file_path, exist_ok=True)

# Define the path to the image file
img_path_in = os.path.join(file_path, 'Image_31.png')

# Load the image using OpenCV
img_raw = cv2.imread(img_path_in)

# Check if the image exists
if img_raw is None:
    raise FileNotFoundError(f'Image not found at {img_path_in}')

# Convert the image from BGR to RGB color format
img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)

# Get image dimensions
img_height, img_width, _ = img_rgb.shape

# Load a pre-trained custom YOLO model.
model = YOLO(f'{project_folder}/YOLO/Results/Stand_2_Dataset_v1/train_fb_True/weights/best.pt')

# Predict (test) the model on a test dataset.
results = model.predict(source=img_path_in, imgsz=640, conf=0.5, iou=0.7)

# Define a dictionary for class_id to color mapping
class_colors = {0: (0, 255, 255), 1: (255, 0, 0)}  # Yellow and Blue in BGR

# If objects are detected in the image
if results[0].boxes.shape[0] >= 1:
    class_id = results[0].boxes.cls.cpu().numpy(); b_box = results[0].boxes.xywhn.cpu().numpy()
    conf = results[0].boxes.conf.cpu().numpy()

    for _, (class_id_i, b_box_i, conf_i) in enumerate(zip(class_id, b_box, conf)):
        x_center, y_center, width, height = b_box_i

        # Convert YOLO coordinates to pixel coordinates
        x_c = int(x_center * img_width); y_c = int(y_center * img_height)
        w = int(width * img_width); h = int(height * img_height)

        # Calculate the top-left corner of the bounding box
        x_top_left = int(x_c - w / 2); y_top_left = int(y_c - h / 2)
        x_bottom_right = int(x_c + w / 2); y_bottom_right = int(y_c + h / 2)

        # Get the bounding box color
        class_id_color = class_colors.get(class_id_i, (0, 255, 0))  # Green as default

        # Draw the bounding box
        cv2.rectangle(img_raw, (x_top_left, y_top_left), (x_bottom_right, y_bottom_right), class_id_color, 2)

        # Put class ID and confidence score
        label = f'Class {int(class_id_i)}: {conf_i:.2f}'
        cv2.putText(img_raw, label, (x_top_left, y_top_left - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, class_id_color, 2)

# Define output image path
output_path = os.path.join(f'../Data/Stand_2/Eval/', 'Image_31_result.png')

# Save the image with bounding boxes
cv2.imwrite(output_path, img_raw)

print(f'Result saved at: {output_path}')